In [9]:
import cv2
import glob
import random
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

In [10]:
metascore = []

In [22]:
emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]
#emotions = ["happy", "surprise"]

In [28]:
clf = GradientBoostingClassifier(criterion='mae',
                                 max_depth=3,
                                 max_features=73,
                                 min_samples_leaf=5,
                                 min_samples_split=13,
                                 n_estimators=200
                                )

In [29]:
def get_files(emotion):
    files = glob.glob("base\dataset\\{}\\*".format(emotion))
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)]
    test = files[-int(len(files) * 0.2):]
    return training, test

In [30]:
def make_sets():
    training_X = []
    training_y = []
    test_X = []
    test_y = []

    for emotion in emotions:

        training, test = get_files(emotion)

        for item in training:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            training_X.append(gray)
            training_y.append(emotions.index(emotion))

        for item in test:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_X.append(gray)
            test_y.append(emotions.index(emotion))

    training_X = np.array(training_X)
    training_y = np.array(training_y)
    test_X = np.array(test_X)
    test_y = np.array(test_y)

    return training_X, training_y, test_X, test_y

In [31]:
training_X, training_y, test_X, test_y = make_sets()

training_X = training_X.reshape(training_X.shape[0], 350 * 350)
test_X = test_X.reshape(test_X.shape[0], 350 * 350)

In [32]:
for i in range(0, 10):
    clf = clf.fit(training_X, training_y)
    predict = clf.predict(test_X)
    score = clf.score(test_X, test_y)
    metascore.append(score)

metascore = np.array(metascore)
    
print ('\033[1m Score: ' + '\033[1;34m {} \n'.format(metascore.mean()*100) + '\033[0m')
print ('\033[1m Margin of Erro: ' + '\033[1;34m {} \n'.format(metascore.std()*100) + '\033[0m')

metascore = []

 Score:  74.72727272727273 

 Margin of Erro:  0.9928059664216312 



In [ ]:
cm = confusion_matrix(test_y, predict, labels=[0, 1, 2, 3, 4, 5, 6, 7])
#cm = confusion_matrix(test_y, predict)

print('\033[1m Matriz de confusao:\n\n' + '\033[0m {} \n'.format(cm))

In [13]:
param_dist = {"max_depth" :  [10, 20, 30],
               "max_features" : sp_randint(2, 10),
               "min_samples_split" : sp_randint(8, 14),
               "min_samples_leaf" : sp_randint(20, 28),
               "n_estimators" : [400, 500, 600],
               "criterion" : ["mae", "mse"]
              }

grid = RandomizedSearchCV(estimator=clf, param_distributions = param_dist, n_iter=100)
grid.fit(training_X, training_y)

y_true, y_pred = test_y, grid.predict(test_X)
bs = grid.best_score_
bp = grid.best_params_
cr = classification_report(y_true, y_pred)

print('\033[1m Best Score: ' + '\033[1;34m {}\n'.format(bs) + '\033[0m')
print('\033[1m Best Params:\n\n' + '\033[0m {}\n'.format(bp))
print('\033[1m Classification Report:\n\n' + '\033[0m {}'.format(cr))

 Best Score:  0.6998087954110899

 Best Params:

 {'criterion': 'mse', 'max_depth': 10, 'max_features': 9, 'min_samples_leaf': 20, 'min_samples_split': 10, 'n_estimators': 600}

 Classification Report:

              precision    recall  f1-score   support

          0       0.68      0.98      0.81        65
          1       0.00      0.00      0.00         9
          2       0.00      0.00      0.00         3
          3       0.83      0.42      0.56        12
          4       0.00      0.00      0.00         5
          5       0.83      0.77      0.80        13
          6       0.00      0.00      0.00         5
          7       1.00      0.94      0.97        16

avg / total       0.63      0.73      0.66       128



C:\Users\hermi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
accuracy = accuracy_score(predict, test_y)
    
print("\n\033[1m Acuracy: \033[1;34m %1.2f%% \033[0m" % accuracy)